<a href="https://colab.research.google.com/github/amitaipat-create/Session2/blob/main/Copy_of_Mission_2_Practice_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HTTP/API Tools with Authentication and Web Search Tools

In [1]:
%pip install langchain==0.3.27 langchain-openai==0.3.29 langchain-community==0.3.27 tavily-python pydantic==2.11.10 python-dotenv==1.0.1 requests --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.1 MB/s eta 0:00:00


In [2]:
import os
import getpass
import json
import time
from dotenv import load_dotenv
from typing import Optional, Dict, Any
import requests

# Load environment variables from .env file (for local development)
load_dotenv()

# OpenAI API Key
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ··········


In [3]:
# TAVILY API Key
if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your Tavily API key: ")

Enter your Tavily API key: ··········


In [4]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.output_parsers import StrOutputParser

# For Colab/Jupyter display (optional)
try:
    from IPython.display import display, Markdown
except ImportError:
    def display(x): print(x)
    def Markdown(x): return x

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

# ============================================================================
# CONCEPT 1: Building HTTP/External API Tool
# Learning: Create a generic REST client tool that agents can use
# This is the foundation for integrating with any external service
# ============================================================================

@tool
def call_api(url: str, method: str = "GET", body: Optional[Dict[str, Any]] = None) -> str:
    """Call any REST API endpoint

    Args:
        url: The API endpoint URL
        method: HTTP method (GET or POST)
        body: Optional request body (for POST)

    Returns:
        JSON response as string
    """
    try:
        if method.upper() == "GET":
            response = requests.get(url)
        elif method.upper() == "POST":
            response = requests.post(url, json=body)
        else:
            return f"Error: Only GET and POST supported"

        response.raise_for_status()
        return json.dumps(response.json(), indent=2)
    except Exception as e:
        return f"Error: {str(e)}"

print("\nBuilding HTTP API Tool...")
print("HTTP API Tool ready")

# Test the tool
print("\nTesting API tool with a public API...")
result = call_api.invoke({
    "url": "https://jsonplaceholder.typicode.com/posts/1",
    "method": "GET"
})
print("API Response:\n")
display(Markdown(f"```json\n{result}\n```"))

print("\n✓ Success! HTTP API tool is working!")


Building HTTP API Tool...
HTTP API Tool ready

Testing API tool with a public API...
API Response:



```json
{
  "userId": 1,
  "id": 1,
  "title": "sunt aut facere repellat provident occaecati excepturi optio reprehenderit",
  "body": "quia et suscipit\nsuscipit recusandae consequuntur expedita et cum\nreprehenderit molestiae ut ut quas totam\nnostrum rerum est autem sunt rem eveniet architecto"
}
```


✓ Success! HTTP API tool is working!


In [5]:
# EXERCISE 1: Try calling different APIs
# Test with: "https://api.github.com/users/octocat" (GET)
# Test with: "https://jsonplaceholder.typicode.com/posts" (POST with body: {"title": "Test", "body": "Hello", "userId": 1})
# See how the tool handles different endpoints and methods!





In [6]:
# CONCEPT 2: HTTP API Tool with Authentication
# Learning: Handle API keys and authentication securely
# Essential for production agents that call protected APIs

In [7]:
# First, let's set up a mock API server for testing
# In Colab, run this in a separate cell to create the server file:

%%writefile mock_api_server.py
from flask import Flask, request, jsonify

app = Flask(__name__)
VALID_API_KEY = "demo_key_12345"

@app.route('/api/data', methods=['GET'])
def get_data():
    auth_header = request.headers.get('Authorization', '')
    if not auth_header.startswith('Bearer '):
        return jsonify({"error": "Missing Authorization header"}), 401
    api_key = auth_header.replace('Bearer ', '')
    if api_key != VALID_API_KEY:
        return jsonify({"error": "Invalid API key"}), 401
    return jsonify({"status": "success", "message": "Authenticated!", "data": {"items": [1, 2, 3]}})

@app.route('/api/data', methods=['POST'])
def post_data():
    auth_header = request.headers.get('Authorization', '')
    if not auth_header.startswith('Bearer '):
        return jsonify({"error": "Missing Authorization header"}), 401
    api_key = auth_header.replace('Bearer ', '')
    if api_key != VALID_API_KEY:
        return jsonify({"error": "Invalid API key"}), 401
    return jsonify({"status": "success", "received": request.json or {}})

if __name__ == '__main__':
    print("Server running on http://127.0.0.1:5000")
    app.run(host='127.0.0.1', port=5000, debug=False)

Writing mock_api_server.py


In [8]:
import subprocess
import threading

def run_server():
    subprocess.run(['python', 'mock_api_server.py'])

server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()
import time
time.sleep(2)
print("Mock API server started!")
print("Valid API key: demo_key_12345")

Mock API server started!
Valid API key: demo_key_12345


In [9]:
# For local testing, the server file is already created
# Just import and run it
VALID_API_KEY = "demo_key_12345"

@tool
def call_authenticated_api(url: str, api_key: str, method: str = "GET",
                          body: Optional[Dict[str, Any]] = None) -> str:
    """Call an API with authentication header

    Args:
        url: API endpoint
        api_key: API key for authentication
        method: HTTP method (GET or POST)
        body: Optional request body

    Returns:
        API response as JSON string
    """
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    try:
        if method.upper() == "GET":
            response = requests.get(url, headers=headers)
        elif method.upper() == "POST":
            response = requests.post(url, headers=headers, json=body)
        else:
            return f"Error: Only GET and POST supported"

        response.raise_for_status()
        return json.dumps(response.json(), indent=2)
    except Exception as e:
        return f"Error: {str(e)}"

print("\nBuilding authenticated API tool...")
print("Authenticated API tool ready")


Building authenticated API tool...
Authenticated API tool ready


In [10]:
# Test with correct API key
print("\nTesting with CORRECT API key...")
result = call_authenticated_api.invoke({
    "url": "http://127.0.0.1:5000/api/data",
    "api_key": VALID_API_KEY,
    "method": "GET"
})
print("API Response:\n")
display(Markdown(f"```json\n{result}\n```"))


Testing with CORRECT API key...
API Response:



```json
{
  "data": {
    "items": [
      1,
      2,
      3
    ]
  },
  "message": "Authenticated!",
  "status": "success"
}
```

In [11]:
# Test with wrong API key
print("\nTesting with WRONG API key...")
result_wrong = call_authenticated_api.invoke({
    "url": "http://127.0.0.1:5000/api/data",
    "api_key": "wrong_key",
    "method": "GET"
})
print("API Response:\n")
display(Markdown(f"```json\n{result_wrong}\n```"))


Testing with WRONG API key...
API Response:



```json
Error: 401 Client Error: UNAUTHORIZED for url: http://127.0.0.1:5000/api/data
```

In [12]:
# ============================================================================
# CONCEPT 3: Real Web Search Tool
# Learning: Build a real web search tool using Tavily API
# This gives agents the ability to search the internet for current information
# ============================================================================

In [13]:
@tool
def search_web(query: str, max_results: int = 3) -> str:
    """Search the web for current information using Tavily API

    Args:
        query: Search query
        max_results: Maximum number of results to return

    Returns:
        Formatted search results
    """
    try:
        tavily_key = os.environ.get("TAVILY_API_KEY")
        if not tavily_key:
            return "Error: TAVILY_API_KEY not found. Get free key from https://app.tavily.com"

        from tavily import TavilyClient
        client = TavilyClient(api_key=tavily_key)
        response = client.search(query, max_results=max_results)

        formatted_results = []
        for i, result in enumerate(response.get('results', [])[:max_results], 1):
            formatted_results.append(
                f"{i}. **{result.get('title', 'No title')}**\n"
                f"   {result.get('content', 'No description')[:200]}...\n"
                f"   URL: {result.get('url', 'No URL')}"
            )
        return "\n\n".join(formatted_results) if formatted_results else "No results found"

    except Exception as e:
        return f"Search error: {str(e)}"

print("\nBuilding real web search tool...")
print("Web search tool ready")


Building real web search tool...
Web search tool ready


In [14]:
print("\nTesting web search tool...")
result_search = search_web.invoke({
    "query": "latest AI news",
    "max_results": 3
})
print("Search Results:\n")
display(Markdown(f"```markdown\n{result_search}\n```"))
print("\n✓ Web search tool tested successfully!")


Testing web search tool...
Search Results:



```markdown
1. **The latest AI news we announced in October - Google Blog**
   Gemini Enterprise is now the "front door" for Google AI in the workplace. New AI security features protect against scams, plus Google Home gets...
   URL: https://blog.google/technology/ai/google-ai-updates-october-2025/

2. **Artificial Intelligence - Latest AI News and Analysis - WSJ.com**
   The latest artificial intelligence news coverage focusing on the technology, tools and the companies building AI technology....
   URL: https://www.wsj.com/tech/ai?gaa_at=eafs&gaa_n=AWEtsqfsk7A34Zm78yJwT7SubdsseDsje3bdEAuxVAM7_dC5vrYvUjMDYVeE&gaa_ts=6921bc2a&gaa_sig=582Nne-_gAzTdguMApr6rV2uYR-Ld3KzpP7ZbSmpown__Tq7ZdbrNtKcEV7A0AV-fWMDiSg3eOsyVZExWUljxw%3D%3D

3. **The Latest AI News and AI Breakthroughs that Matter Most: 2025**
   Summary: Xiaomi has announced a next-gen AI voice model optimized for in-car and smart home experiences. The model features faster response times, offline...
   URL: https://www.crescendo.ai/news/latest-ai-news-and-updates
```


✓ Web search tool tested successfully!


In [15]:
# Run the tools with Agent
# Step 1: Collect all tools
tools = [search_web, call_authenticated_api]

In [16]:
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate

# 1. Define a prompt template for the agent
agent_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Use the available tools to answer questions and fulfill requests."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

# 2. Collect all tools
tools = [search_web, call_authenticated_api, call_api]

# 3. Create the agent
agent = create_tool_calling_agent(llm, tools, agent_prompt)

# 4. Create an AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

print("\nAgent setup complete. Ready to invoke with examples.")


Agent setup complete. Ready to invoke with examples.


In [17]:
# Invoke with examples
print("\n--- Example 1: Web search ---")
response1 = agent_executor.invoke({"input": "What are the latest AI news headlines?"})
display(Markdown(f"**Agent Response:**\n{response1['output']}"))


--- Example 1: Web search ---


> Entering new AgentExecutor chain...

Invoking: `search_web` with `{'query': 'latest AI news', 'max_results': 5}`


1. **AI News | Latest AI News, Analysis & Events**
   AI News delivers the latest updates in artificial intelligence, machine learning, deep learning, enterprise AI, and emerging tech worldwide....
   URL: https://www.artificialintelligence-news.com/

2. **AI News | Latest Headlines and Developments | Reuters**
   Explore the latest artificial intelligence news with Reuters - from AI breakthroughs and technology trends to regulation, ethics, business and global...
   URL: https://www.reuters.com/technology/artificial-intelligence/

3. **Artificial Intelligence - Latest AI News and Analysis - WSJ.com**
   The latest artificial intelligence news coverage focusing on the technology, tools and the companies building AI technology....
   URL: https://www.wsj.com/tech/ai?gaa_at=eafs&gaa_n=AWEtsqc1wMTEEMa1OSC2irk7pLv3Et8sZ0Q0xPy8qLwTiWw_cI2uLkfA

**Agent Response:**
Here are some of the latest AI news headlines:

1. **AI News | Latest AI News, Analysis & Events** - AI News delivers updates in artificial intelligence, machine learning, deep learning, and emerging tech worldwide. [Read more](https://www.artificialintelligence-news.com/)

2. **AI News | Latest Headlines and Developments | Reuters** - Explore the latest artificial intelligence news, including breakthroughs, technology trends, regulation, and ethics. [Read more](https://www.reuters.com/technology/artificial-intelligence/)

3. **Artificial Intelligence - Latest AI News and Analysis - WSJ.com** - Coverage focusing on technology, tools, and companies building AI technology. [Read more](https://www.wsj.com/tech/ai?gaa_at=eafs&gaa_n=AWEtsqc1wMTEEMa1OSC2irk7pLv3Et8sZ0Q0xPy8qLwTiWw_cI2uLkfAnjt8&gaa_ts=6921bc2c&gaa_sig=fv3Ouj6wALBuPO15rz7SpQ_tW7xE81fuMIbxZgvJ2njs17KxKNo5YhCif_j2CmaOpwWziZd5UANgxHDQ7VFQ2g%3D%3D)

4. **The Latest AI News and AI Breakthroughs that Matter Most: 2025** - New research shows AI can accurately screen for diabetic retinopathy before symptoms arise. [Read more](https://www.crescendo.ai/news/latest-ai-news-and-updates)

5. **Artificial Intelligence | Latest News, Photos & Videos | WIRED** - Find the latest AI news, related science and technology articles, photos, and videos. [Read more](https://www.wired.com/tag/artificial-intelligence/)

Feel free to explore these links for more detailed information!

In [18]:
print("\n--- Example 2: Call authenticated API with correct key ---")
response2 = agent_executor.invoke({"input": f"Call the authenticated API at http://127.0.0.1:5000/api/data with a GET method using the API key {VALID_API_KEY}."})
display(Markdown(f"**Agent Response:**\n```json\n{response2['output']}\n```"))


--- Example 2: Call authenticated API with correct key ---


> Entering new AgentExecutor chain...

Invoking: `call_authenticated_api` with `{'url': 'http://127.0.0.1:5000/api/data', 'api_key': 'demo_key_12345', 'method': 'GET'}`


{
  "data": {
    "items": [
      1,
      2,
      3
    ]
  },
  "message": "Authenticated!",
  "status": "success"
}The API call was successful! Here are the details:

- **Status:** success
- **Message:** Authenticated!
- **Data Items:** [1, 2, 3] 

If you need any further information or actions, let me know!

> Finished chain.


**Agent Response:**
```json
The API call was successful! Here are the details:

- **Status:** success
- **Message:** Authenticated!
- **Data Items:** [1, 2, 3] 

If you need any further information or actions, let me know!
```

In [19]:
print("\n--- Example 3: Call authenticated API with wrong key ---")
response3 = agent_executor.invoke({"input": "Call the authenticated API at http://127.0.0.1:5000/api/data with a GET method using the API key 'wrong_key'. What is the error?"})
display(Markdown(f"**Agent Response:**\n```json\n{response3['output']}\n```"))


--- Example 3: Call authenticated API with wrong key ---


> Entering new AgentExecutor chain...

Invoking: `call_authenticated_api` with `{'url': 'http://127.0.0.1:5000/api/data', 'api_key': 'wrong_key', 'method': 'GET'}`


Error: 401 Client Error: UNAUTHORIZED for url: http://127.0.0.1:5000/api/dataThe error returned is a "401 Client Error: UNAUTHORIZED." This indicates that the API key provided ('wrong_key') is incorrect or not authorized to access the requested resource.

> Finished chain.


**Agent Response:**
```json
The error returned is a "401 Client Error: UNAUTHORIZED." This indicates that the API key provided ('wrong_key') is incorrect or not authorized to access the requested resource.
```

In [20]:
print("\n--- Example 4: Call public API ---")
response4 = agent_executor.invoke({"input": "Get information about post ID 5 from https://jsonplaceholder.typicode.com/posts/5"})
display(Markdown(f"**Agent Response:**\n```json\n{response4['output']}\n```"))

print("\nAll agent examples invoked successfully!")


--- Example 4: Call public API ---


> Entering new AgentExecutor chain...

Invoking: `call_api` with `{'url': 'https://jsonplaceholder.typicode.com/posts/5'}`


{
  "userId": 1,
  "id": 5,
  "title": "nesciunt quas odio",
  "body": "repudiandae veniam quaerat sunt sed\nalias aut fugiat sit autem sed est\nvoluptatem omnis possimus esse voluptatibus quis\nest aut tenetur dolor neque"
}Here is the information about post ID 5:

- **User ID**: 1
- **Post ID**: 5
- **Title**: nesciunt quas odio
- **Body**: 
  ```
  repudiandae veniam quaerat sunt sed
  alias aut fugiat sit autem sed est
  voluptatem omnis possimus esse voluptatibus quis
  est aut tenetur dolor neque
  ```

> Finished chain.


**Agent Response:**
```json
Here is the information about post ID 5:

- **User ID**: 1
- **Post ID**: 5
- **Title**: nesciunt quas odio
- **Body**: 
  ```
  repudiandae veniam quaerat sunt sed
  alias aut fugiat sit autem sed est
  voluptatem omnis possimus esse voluptatibus quis
  est aut tenetur dolor neque
  ```
```


All agent examples invoked successfully!


In [21]:
# EXERCISE 1: HTTP/API Tool (call_api) - Different GET Use Cases
# Task: Use the `call_api` tool to get information about a different public GitHub user (e.g., 'google' or 'microsoft').
# Then, try to use it to fetch data from an API that might require a specific header or query parameter if you can find one.
# Example Hint: `call_api.invoke({"url": "https://api.github.com/users/google", "method": "GET"})`


# EXERCISE 2: Authenticated API Tool (call_authenticated_api) - POST Request
# Task: Perform a POST request to the mock authenticated API with a valid key.
# URL: "http://127.0.0.1:5000/api/data"
# API Key: VALID_API_KEY (from notebook variable)
# Body: `{"action": "create", "item": "user_log_entry", "timestamp": "2024-07-29T10:00:00Z"}`
# Verify the response indicates success and includes the received body.


# EXERCISE 3: Web Search Tool (search_web) - Specific Topic & More Results
# Task: Use the `search_web` tool to find "the history of artificial intelligence" and request 5 results.
# Observe how the `max_results` parameter changes the output.
# Example Hint: `search_web.invoke({"query": "the history of artificial intelligence", "max_results": 5})`


# EXERCISE 4: Agent Integration - Combining Tools and Information Synthesis
# Task: Ask the agent a question that requires both web search and synthesizing information.
# Example Prompt: "What is the current status of renewable energy adoption in Germany, and what are 2 leading companies in that sector?"
# Observe how the agent uses `search_web` and structures its response.
# Example Hint: `agent_executor.invoke({"input": "What is the current status of renewable energy adoption in Germany, and what are 2 leading companies in that sector?"})`


# EXERCISE 5: Agent Integration - Authenticated POST Action
# Task: Ask the agent to perform a POST request to the authenticated API using the `VALID_API_KEY`.
# Example Prompt: "Send a POST request to the authenticated API at http://127.0.0.1:5000/api/data with the API key 'demo_key_12345' and a body that says '{"event": "new_user", "user_id": 123}'"
# Verify that the agent correctly identifies and calls the `call_authenticated_api` tool with the provided details.